In [83]:
from pathlib import Path
import pandas as pd

In [84]:
pwd = Path('.')

In [85]:
cases = [ 
    { 'f': f, 'id': f.name, 
      'fs': list(f.glob('**/*.tsv'))[0]
    } 
          for f in pwd.iterdir() if ( f.is_dir() and f.name[:2] != '__' ) ]

In [86]:
## DOWNLOAD CHECKPOINT FILES EXIST IN INCOMPLETE DOWNLOADS

In [87]:
count = 0
for i in cases:
    if len(list(i['f'].glob('**/*.tsv'))) < 1:
        print(i)

In [88]:
cases[0]

{'f': PosixPath('56be281f-e85c-4dcb-9d75-cd9ae3430bbe'),
 'id': '56be281f-e85c-4dcb-9d75-cd9ae3430bbe',
 'fs': PosixPath('56be281f-e85c-4dcb-9d75-cd9ae3430bbe/92e7a9c0-ea88-419e-9afa-e7db5c8967b5.rna_seq.augmented_star_gene_counts.tsv')}

In [89]:
cases_count = len(cases)

In [91]:
final_tables = []
k = 0
for k in range(cases_count):
    try:
        data = cases[k]
        a = data['fs']
        df = pd.read_csv(data['fs'], sep='\t', skiprows=1)
        df = df[['gene_name', 'gene_type', 'tpm_unstranded']].dropna()
        df = df.query("gene_type == 'protein_coding'").drop('gene_type', axis=1)
        print(k, "======") if k%100 == 0 else None
        #print(df.head())
        df['file_id'] = data['id']
        #print(df['case_id'].iloc[0])
        df_T = pd.pivot_table(df, values=['tpm_unstranded'], index='file_id', columns='gene_name')
        
        final_tables.append(df_T)
    except Exception as e:
        print(f'{e} == {a}')
        
tpm_df = pd.concat(final_tables)
#final_df.index.name = None
tpm_df.columns.name = None
#final_df = final_df.reset_index()
tpm_df['tpm_unstranded'].columns.name = None


0 ======
100 ======
200 ======
300 ======
No columns to parse from file == 36e4ffb0-0bab-4035-88f1-a181b195fce1/d78126ec-68b1-4d77-bf7a-19a47678a5fb.rna_seq.augmented_star_gene_counts.tsv
No columns to parse from file == 8849e5bf-ff35-4cbf-972c-01fb92201a73/feb7140e-d822-4462-8dfa-7a63805ac740.rna_seq.augmented_star_gene_counts.tsv
400 ======
500 ======
600 ======
700 ======
800 ======
900 ======
1000 ======
No columns to parse from file == a4589532-53e0-4025-b8d3-e11e79e1fc9c/9de4575c-0d10-4548-8881-fd4a5a46a5ac.rna_seq.augmented_star_gene_counts.tsv
1100 ======
"None of [Index(['gene_name', 'gene_type', 'tpm_unstranded'], dtype='object')] are in the [columns]" == .ipynb_checkpoints/gdc_sample_sheet.2025-12-19-checkpoint.tsv
1200 ======


NameError: name 'final_df' is not defined

In [92]:
tpm_df = tpm_df['tpm_unstranded']
tpm_df.to_csv('__cleaned/tpm_unstranded.tsv', sep='\t')

## READ FINAL DATA

## COMBINE CLINICAL DATA

In [3]:
import pandas as pd
from pathlib import Path
import json

In [4]:
clinical = pd.read_csv('__clinical/clinical.tsv', sep='\t')

/tmp/ipykernel_878/2935981137.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  clinical = pd.read_csv('__clinical/clinical.tsv', sep='\t')


In [5]:
read = pd.read_csv('__cleaned/tpm_unstranded.tsv', sep='\t')

In [6]:
read.head(2)

,file_id,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
0,56be281f-e85c-4dcb-9d75-cd9ae3430bbe,0.2784,0.0064,185.8713,0.0691,0.0,21.2918,0.1043,33.0771,10.2845,...,8.9504,41.2676,7.7594,28.7315,21.2916,2.4903,24.4805,177.3887,13.6639,24.6079
1,d657ed9a-fd92-493c-ae10-120c8b0c58d7,0.5090,0.0000,101.9758,0.0692,0.0,5.4224,0.1237,23.7173,8.8077,...,12.8709,26.1977,7.5701,25.8253,12.0619,0.8204,35.0268,75.7216,14.1335,29.3801


### METADATA

In [7]:
#metadata = pd.read_json('__raw/metadata.repository.2025-12-19.json')
with open('__raw/metadata.repository.2025-12-19.json', 'r') as f:
    metadata_json = json.load(f)
ls = [ len(x['associated_entities']) for x in metadata_json]
max(ls)

1

In [8]:

case_ids = []
file_ids = []
for x in metadata_json:
    case_ids.append(x['associated_entities'][0]['case_id'])
    file_ids.append(x['file_id'])
len(case_ids), len(file_ids)

(1231, 1231)

In [9]:
metadata_dict = {
    'case_id': case_ids,
    'file_id': file_ids
}
case_file_map = pd.DataFrame(metadata_dict)

In [10]:
case_file_map.head()

,case_id,file_id
0,878f975b-94fd-4d69-b7e7-1ed3ac2ee438,744a6d3d-b666-49aa-8d26-47f34e3d1eb5
1,e4fc0909-f284-4471-866d-d8967b6adcbc,4ecc1f1a-8ff4-4552-a5e8-7a9652b6d1d5
2,e5aae05a-478e-4a55-a27c-12b2b4be302a,1ace2a0c-773d-45b5-8fd6-968c88731bbb
3,16fc3677-0393-4ed1-ad3f-c8355f056369,2d5b0962-b5c4-4482-9f28-47e4dcdb6df6
4,33fe5833-d7a0-43d7-a03d-56985ea448a6,84cff855-d17a-4c09-b432-53893ae69c1e


In [11]:
df = pd.merge(pd.merge(read, case_file_map, on='file_id'), clinical, left_on='case_id', right_on='cases.case_id')
df.head()

,file_id,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,...,treatments.treatment_duration,treatments.treatment_effect,treatments.treatment_effect_indicator,treatments.treatment_frequency,treatments.treatment_id,treatments.treatment_intent_type,treatments.treatment_or_therapy,treatments.treatment_outcome,treatments.treatment_outcome_duration,treatments.treatment_type
0,56be281f-e85c-4dcb-9d75-cd9ae3430bbe,0.2784,0.0064,185.8713,0.0691,0.0,21.2918,0.1043,33.0771,10.2845,...,'--,'--,'--,'--,3234723f-c2c7-4b8b-a889-8ab985f2bf8d,'--,no,'--,'--,"Pharmaceutical Therapy, NOS"
1,56be281f-e85c-4dcb-9d75-cd9ae3430bbe,0.2784,0.0064,185.8713,0.0691,0.0,21.2918,0.1043,33.0771,10.2845,...,'--,'--,'--,'--,e0a284cf-2fd8-46a1-a7af-7baf53c1b1cc,'--,no,'--,'--,"Radiation Therapy, NOS"
2,56be281f-e85c-4dcb-9d75-cd9ae3430bbe,0.2784,0.0064,185.8713,0.0691,0.0,21.2918,0.1043,33.0771,10.2845,...,'--,'--,'--,'--,0b01cfef-7b14-4865-8a3d-6f61e2936e82,First-Line Therapy,yes,'--,'--,"Surgery, NOS"
3,56be281f-e85c-4dcb-9d75-cd9ae3430bbe,0.2784,0.0064,185.8713,0.0691,0.0,21.2918,0.1043,33.0771,10.2845,...,'--,'--,'--,'--,10db805e-6243-5060-b4d0-8ead37a3f11c,'--,yes,Complete Response,'--,"Radiation, External Beam"
4,56be281f-e85c-4dcb-9d75-cd9ae3430bbe,0.2784,0.0064,185.8713,0.0691,0.0,21.2918,0.1043,33.0771,10.2845,...,'--,'--,'--,'--,b3182d58-86d4-41cc-a03e-a2ed295f6cef,'--,yes,Treatment Ongoing,'--,Hormone Therapy


In [14]:
#list(df.columns)

In [15]:
df.to_csv('__cleaned/final_tpm_unstranded.tsv', sep='\t')